In [36]:
import pandas as pd
from recipe_scrapers import scrape_me
import parse_ingredient

In [15]:
scraper = scrape_me('https://www.allrecipes.com/recipe/158968/spinach-and-feta-turkey-burgers/')

In [23]:
scraper.total_time()

35

In [33]:
ingredients = scraper.ingredients()

In [38]:
ingredients

['2 eggs, beaten',
 '2 cloves garlic, minced',
 '4 ounces feta cheese',
 '1 (10 ounce) box frozen chopped spinach, thawed and squeezed dry',
 '2 pounds ground turkey']

In [60]:
data = []
for ingredient in ingredients:
    print(ingredient)
    info = parse_ingredient.parse(ingredient)
    info = info.as_dict()
    data.append((info["product"], info["quantity"], info["unit"], info["usda_info"]["category"]))

2 eggs, beaten


InsufficientQuotaError: You have insufficient quota to complete this request. To continue parsing ingredients, purchase a Zestful plan from https://zestfuldata.com

In [55]:
data = parse_ingredient.parse(ingredients[2])
info = data.as_dict()

In [56]:
info

{'confidence': 0.9916780000000001,
 'product': 'feta cheese',
 'productSizeModifier': None,
 'quantity': 4.0,
 'unit': 'ounce',
 'preparationNotes': None,
 'usda_info': {'category': 'Dairy and Egg Products',
  'description': 'Cheese, feta',
  'fdcId': '173420',
  'matchMethod': 'exact'}}